# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
df = pd.read_csv("./data/AviationData.csv", encoding='ISO-8859-1', low_memory=False)

In [3]:
print(df.isnull().sum())

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
Publication.Date          13771
dtype: i

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [8]:
df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [13]:
df[["Aircraft.Category", "Make", "Model", "Amateur.Built", "Event.Date", "Aircraft.damage"]].info()

# checking diff categories present
df["Aircraft.Category"].unique()
df["Make"].unique()

# fixing date
df["Event.Date"] = pd.to_datetime(df["Event.Date"], errors='coerce')
# filter out rows where date isnt there
df = df[df["Event.Date"].notnull()]

# filter out rows where make model is missing
df = df[df["Make"].notnull() & df["Model"].notnull()]

<class 'pandas.core.frame.DataFrame'>
Index: 88777 entries, 0 to 88888
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Aircraft.Category  32245 non-null  object        
 1   Make               88777 non-null  object        
 2   Model              88777 non-null  object        
 3   Amateur.Built      88678 non-null  object        
 4   Event.Date         88777 non-null  datetime64[ns]
 5   Aircraft.damage    85605 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 4.7+ MB


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [18]:
injured = ["Total.Fatal.Injuries", "Total.Serious.Injuries", "Total.Minor.Injuries", "Total.Uninjured"]

for i in injured:
    df[i] = pd.to_numeric(df[i]).fillna(0)

df["total_people"] = df["Total.Fatal.Injuries"] + df["Total.Serious.Injuries"] + df["Total.Minor.Injuries"] + df["Total.Uninjured"]
# now remove row where 0
df = df[df["total_people"] > 0]

df["fatalOrSerious"] = (df["Total.Fatal.Injuries"] + df["Total.Serious.Injuries"]) / df["total_people"]
df["fatalOrSerious"]

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
88882    0.5
88883    1.0
88884    1.0
88886    0.0
88888    0.5
Name: fatalOrSerious, Length: 87498, dtype: float64

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [20]:
df["Aircraft.damage"] = df["Aircraft.damage"].str.lower()
df["Aircraft.damage"].value_counts(dropna=False)
df["aircraftDestroyed"] = df["Aircraft.damage"].apply(
    lambda x: 1 if x == "DESTROYED" else 0
)

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [21]:
df["Make"] = df["Make"].str.lower().str.strip()

df["Make"] = df["Make"].replace({
    "BOEING COMMERCIAL AIRPLANE CO.": "BOEING",
    "BOEING COMPANY": "BOEING",
    "MCDONNELL DOUGLAS CORPORATION": "MCDONNELL-DOUGLAS",
    "MCDONNELL DOUGLAS": "MCDONNELL-DOUGLAS",
    "CESSNA AIRCRAFT COMPANY": "CESSNA",
    "CESSNA AIRCRAFT": "CESSNA",
    "CESSNA AIRCRAFT CO": "CESSNA",
    "PIPER AIRCRAFT": "PIPER",
    "AIRBUS INDUSTRIES": "AIRBUS",
    "PIPER AIRCRAFT INC.": "PIPER",
    "AIRBUS INDUSTRIE": "AIRBUS",
    "LOCKHEED AIRCRAFT CORPORATION": "LOCKHEED",
    "LOCKHEED MARTIN": "LOCKHEED",
    "BEECH AIRCRAFT": "BEECH",
    "BEECH AIRCRAFT CORPORATION": "BEECH",
    "ROBINSON HELICOPTER CO.": "ROBINSON",
    "ROBINSON HELICOPTER": "ROBINSON",
    "ROBINSON HELICOPTER COMPANY": "ROBINSON"
})

df["Make"].value_counts()
df["Make"].value_counts()[df["Make"].value_counts() >= 50]

Make
cessna                         27043
piper                          14828
beech                           5344
bell                            2686
boeing                          2145
                               ...  
lancair                           52
boeing stearman                   51
smith, ted aerostar               51
flight design gmbh                50
grumman american avn. corp.       50
Name: count, Length: 98, dtype: int64

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [22]:
df = df[df["Model"].notnull()]
df["Model"].nunique()
df["AircraftType"] = df["Make"] + "-" + df["Model"]

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [24]:
df["Engine.Type"] = df["Engine.Type"].str.lower().str.strip()
df["Engine.Type"] = df["Engine.Type"].replace({
    "NONE": "NONE",
    "UNK": "UNKNOWN"
})

df["Weather.Condition"] = df["Weather.Condition"].str.lower().str.strip()
df["Weather.Condition"] = df["Weather.Condition"].replace({
    "UNK": "UNKNOWN"
})

df["Broad.phase.of.flight"] = df["Broad.phase.of.flight"].str.title().str.strip()

df["Purpose.of.flight"] = df["Purpose.of.flight"].str.title().str.strip()
df["Purpose.of.flight"] = df["Purpose.of.flight"].replace({
    "Air Race/Show": "Air Race/Show",
    "Air Race Show": "Air Race/Show",
    "Pubs": "Other/Unknown",
    "Asho": "Other/Unknown",
    "Publ": "Other/Unknown"
})

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [25]:
missing = df.isnull().sum()
toDrop = missing[missing > (0.5 * len(df))].index.tolist()

df.drop(columns=toDrop, inplace=True)

df

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,total_people,fatalOrSerious,aircraftDestroyed,AircraftType
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,Fatal(2),destroyed,...,0.0,0.0,unk,Cruise,Probable Cause,NaN,2.0,1.0,0,stinson-108-3
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,Fatal(4),destroyed,...,0.0,0.0,unk,Unknown,Probable Cause,19-09-1996,4.0,1.0,0,piper-PA24-180
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,NaN,NaN,Fatal(3),destroyed,...,0.0,0.0,imc,Cruise,Probable Cause,26-02-2007,3.0,1.0,0,cessna-172M
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,Fatal(2),destroyed,...,0.0,0.0,imc,Cruise,Probable Cause,12-09-2000,2.0,1.0,0,rockwell-112
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,Fatal(1),destroyed,...,0.0,0.0,vmc,Approach,Probable Cause,16-04-1980,3.0,1.0,0,cessna-501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88882,20221222106486,Accident,CEN23LA068,2022-12-21,"Reserve, LA",United States,NaN,NaN,Minor,NaN,...,0.0,1.0,NaN,NaN,NaN,27-12-2022,2.0,0.5,0,grumman american avn. corp.-AA-5B
88883,20221228106502,Accident,GAA23WA046,2022-12-22,"Brasnorte,",Brazil,NaN,NaN,Fatal,NaN,...,0.0,0.0,NaN,NaN,NaN,28-12-2022,1.0,1.0,0,air tractor-AT502
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,Minor,NaN,...,0.0,0.0,NaN,NaN,NaN,29-12-2022,1.0,1.0,0,piper-PA-28-151
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,PAN,PAYSON,Non-Fatal,substantial,...,0.0,1.0,vmc,NaN,NaN,27-12-2022,1.0,0.0,0,american champion aircraft-8GCBC


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [27]:
df.to_csv("data/clean_aviation_data.csv", index=False)